In [1]:
import pandas as pd
import random
from math import factorial as fact
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None
from collections import Counter
import time
import sys
import glob
import re
from itertools import chain
import os
import random

# Draw of cards and table settings

In [2]:
import random

Heads = set(["A","K","Q","J","T","9","8","7","6","5","4","3","2"])
Colors = set(["s","c","h","d"])
Cards = {a+b for a in Heads for b in Colors}
k=100000
k_vilain = 4

c_hero = ["As","Kc"]

def tirage(Cards,k_vilain,Hero):
    Cards_copy = set(Cards)
    vilain_cards=[]
    for elmt in Hero:
        Cards_copy.remove(elmt)

    for i in range(k_vilain):
        c = random.sample(Cards_copy, 2)
        vilain_cards.append(c)
        for elmt in c: 
            Cards_copy.remove(elmt)
    return vilain_cards

def is_in_range(card_r,vilain_cards):
    #card_r : range of cards that we want to assess the probability
    #vilain_cards : list of the cards drawn by the vilain
    S=0
    combo = [a[0][0]+a[1][0] for a in vilain_cards]
    for elmt in card_r:
        if elmt in combo:
            S=1
    return S

def proba_card_vilain(Cards,k_vilain,card_r,c_hero,k):
    #Cards : pack of cards in a standard game
    #k_vilain : number of other players in the table
    #c_hero : cards dealt to hero
    #k : number of iterations
    #card_r : range of cards in format ["AK","QQ",etc..]
    P=0
    for i in range(k):
        a=tirage(Cards,k_vilain,c_hero)
        P+=is_in_range(card_r,a)
    return P/k

c_hero = ["As","Ac"]
k_vilain = 5
card_r = ["AK","AQ","22","33","44"]
k=100000
proba_card_vilain(Cards,k_vilain,card_r,c_hero,k)

0.10343

In [3]:
hand_strength = {"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"T":10,"J":11,"Q":12,"K":13,"A":14}
same_dict = {2:[],3:[],4:[],"s":[],"f":[],'sf':[]}
def draw_in_range(Cards,c_range):
    #we draw two cards for vilain and vilain and hero ranges
    #Cards : remaining card in our pack
    #c_range : range of cards for the draw
    
    #We create a subsample "l" based on vilain range
    l=sub_sample(Cards,c_range)
    #We draw a random hand from the subsamble
    c_draw=random.sample(l, 1)
    for c in c_draw[0]:
        Cards_copy.remove(c)
    return tuple(c_draw)

def sub_sample(Cards,c_range):
    #c_range : range of cards
    #creation of the sample of all hands in a range
    c_range=range_arrange(c_range)
    l1=[(u[0]+c,u[1]+c) for u in c_range for c in Colors if u[2]=="s" if u[0]+c in Cards if u[1]+c in Cards ]
    l2=[(u[0]+c,u[1]+d) for u in c_range for c in Colors for d in Colors if c != d if u[2]=="o" if u[0]+c in Cards if u[1]+d in Cards ]
    return l1+l2

def range_arrange(c_range):
    #We add "o" to pairs if not precisedlen(l)
    return [l+"o" if len(l)==2 else l for l in c_range]

def PF_draws(Cards,h_range,*vilain_range):
    #We draw PF cards to the players in the hand
    h_hand=draw_in_range(Cards,h_range)
    h_vilain = ()
    for r in vilain_range:
        h_vilain += draw_in_range(Cards,r)
    h_vilain += h_hand
    return h_vilain

def B_draw(Cards,i):
    #We draw the three cards of the flop
    c_draw=random.sample(Cards, i)
    for c in c_draw:
        Cards.remove(c)
    return c_draw

def sim(Cards,board,h_range,*v_range):
    #simulate the remaining
    #board : tuple of cards already in the board
    for c in board:
        Cards.remove(c)
    p_cards=PF_draws(Cards,h_range,*v_range)
#     print(p_cards)
    b_cards = board + B_draw(Cards,5-len(board))
#     print(tuple(b_cards))
    for c in board:
        if c in Cards:
            Cards.remove(c)
    h_dict = {"board": tuple(b_cards)}
    i=1
    for p in p_cards:
        h_dict[f"player {i}"]=p
        i+=1
    return h_dict

def h_made(h_dict):
    h_dict_copy = dict(h_dict)
    board = h_dict_copy.pop("board",0)
    player_hand = {key: h_val(item+board) for key,item in h_dict_copy.items()}
    return player_hand

def h_val(hand):
    same_dict = {"c":[],2:[],3:[],4:[],"s":[],"f":[],"sf":[]}
    f=flush(hand)
    sf=straight_flush(hand)
    if f:
        same_dict['f']=f
    if sf:
        same_dict["sf"]=sf
    h=tuple([hand_strength[h[0]] for h in hand])
    c=sorted(list(h),reverse=True)
    same_dict["c"]=c
    if 13 in h:
        h=h+(1,)
    s=straight(h)
    if s:
        same_dict['s']=s[-1]
    t=Counter(h).items()
    for elmt in t:
        if elmt[1]==2 or elmt[1]==3 or elmt[1]==4:
            same_dict[elmt[1]].append(elmt[0])
    return same_dict

def straight(hand):
    c=set(sorted(hand))
    return [c.intersection(set(range(1+i,6+i)))  for i in range(10) if len(c.intersection(set(range(1+i,6+i))))==5]

def flush(hand):
    s=[h for h in hand if 's' in h]
    d=[h for h in hand if 'd' in h]
    c=[h for h in hand if 'c' in h]
    h=[h for h in hand if 'h' in h]
    L=[s,d,c,h]
    for elmt in L:
        if len(elmt)>=5:
            return tuple(elmt)
        
def straight_flush(hand):
    h=flush(hand)
    if h:
        h=tuple([hand_strength[h[0]] for h in h])
        if 13 in h:
            h=h+(1,)
        return straight(h)
    
# def score(p_dict):
#     for players,score in p_dict.items:
#         if score["sf"]:
            


In [11]:
p = [a+a for a in Heads]
q = [a+b for a in Heads for b in Heads if a!=b]
q1=[q+'o' for q in q]
q2=[q+'s' for q in q]
R=p+q1+q2

Cards_copy = set(Cards)

h_range = R
v1_range = R
v2_range = R
v3_range = R
v4_range = R
v5_range = R
board = []
#we transform the range into an ensemble of hands
L=["AKs","AQo","AQs","AA"]
Heads = set(["A","K","Q","J","T","9","8","7","6","5","4","3","2"])
Colors = set(["s","c","h","d"])

i=1
j=0
while i:
    Cards_copy = set(Cards)
#     u=sim(Cards_copy,board,h_range,v1_range,v2_range,v3_range)
    u=sim(Cards_copy,board,h_range,v1_range,v2_range,v3_range)
    x=h_made(u)   
    j+=1
    for key,val in x.items():
        if (val["sf"] or val["f"] or val["s"] or val[4] or (val[3] and val[2])):
            i=0
print(u)
print(j) 
print(x)

{'board': ('Js', '8h', '7h', 'Ts', '5d'), 'player 1': ('8c', 'Jh'), 'player 2': ('5s', '9s'), 'player 3': ('8s', '6h'), 'player 4': ('7s', 'Kd')}
2
{'player 1': {'c': [11, 11, 10, 8, 8, 7, 5], 2: [8, 11], 3: [], 4: [], 's': [], 'f': [], 'sf': []}, 'player 2': {'c': [11, 10, 9, 8, 7, 5, 5], 2: [5], 3: [], 4: [], 's': {7, 8, 9, 10, 11}, 'f': [], 'sf': []}, 'player 3': {'c': [11, 10, 8, 8, 7, 6, 5], 2: [8], 3: [], 4: [], 's': [], 'f': [], 'sf': []}, 'player 4': {'c': [13, 11, 10, 8, 7, 7, 5], 2: [7], 3: [], 4: [], 's': [], 'f': [], 'sf': []}}


In [12]:
def best_combo(hval):
    if hval["sf"]:
        return "STRAIGHT FLUSH :"+str(hval["sf"])
    if hval[4]:
        return "SQUARE :"+str(hval[4])
    if (hval[3] and hval[2]):
        return "FULL :"+str(hval[3])+str(hval[2])
    if hval["s"]:
        return "STRAIGHT :"+str(hval["s"])
    if hval["s"]:
        return "STRAIGHT :"+str(hval["s"])
    if hval[3]:
        return "3 OF A KIND :"+str(hval[3])
    if len(hval[2])>=2:
        return "DOUBLE PAIR :"+str(hval[2])
    if hval[2]:
        return "PAIRE :"+str(hval[2])
    else:
        return "AIR"
    
best_combo(x["player 1"])

'DOUBLE PAIR :[8, 11]'

In [13]:
def long_term_range(n_hand,k_vilain,confidence,k):

    a=set(range(100))
    lim = 0
    count = 0
    X=0
    p0=0
    p1=0
    while p1<confidence:
        p0=p1
        lim +=1
        count=0
        for i in range(k):
            u=[random.sample(a, 1)[0] for j in range(n_hand)]
            c=[i for i in u if i< lim]
            if len(c)>=n_hand*k_vilain:
                count+=1
        p1=count/k
        print(f"Vilain 3bet% in {n_hand} hands == {p_vilain*100}% /// Vilain long term 3Bet% <= {lim} in {round(p0*100,3)} times out of 100)")
    return [lim,round(p0,3)]

n_hand = 8
p_vilain = 0.25
confidence = 0.20
k = 10000
inc=3

long_term_range(n_hand,p_vilain,confidence,k)
        

Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 1 in 0 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 2 in 0.29 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 3 in 1.01 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 4 in 2.14 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 5 in 3.78 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 6 in 6.07 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 7 in 7.75 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 8 in 9.86 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 9 in 13.08 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 10 in 15.56 times out of 100)
Vilain 3bet% in 8 hands == 25.0% /// Vilain long term 3Bet% <= 11 in 19.18 times out of 10

[11, 0.192]